In [1]:
import numpy as np
import pandas as pd
import re
import sys 
from pyspark import SparkConf, SparkContext

from pyspark.sql import SQLContext
from scipy.spatial import distance
from pyspark.sql import Row

conf = SparkConf()
sc = SparkContext(conf = conf)

In [2]:
dataset_path="graph-full.txt" # path to the dataset
iterations = 40 # number of iterations
beta = 0.8

In [3]:
def page_rank(dataset_path, iterations, beta):
    small1 =np.loadtxt(dataset_path) #loads data from text file
# type(small1) 
    small2=np.unique(small1, axis=0) # removes multiple edges
    small=sc.parallelize(small2) #creates an rdd
# small.count()
# small.take(5)
# type(small)
    outdegreeapointsb = small.groupByKey().mapValues(list) #generates a,(b,c,..) where a-> b, a->c, ..
# outdegreeapointsb.take(5)
    outdegapointsblen = outdegreeapointsb.map(lambda x:( x[0], x[1] , 1/len(x[1])))
# counting the length of list and displaying 1/len
# outdegapointsblen.take(5)
# outdegapointsblen.count()
    nodecommaoutdeg = outdegreeapointsb.map(lambda x: (x[0], 1/len(x[1])))
# nodecommaoutdeg.take(5)                                        
#rdd node, 1/outdegree of that node.  This is our first main rdd
    rddapointsb = small.map(lambda x: (x[0], x[1]))
# rddapointsb.count()
# rddapointsb.take(5) 

    joinednode1points2deg1 = rddapointsb.join(nodecommaoutdeg)#two rdds are now joined
# joinednode1points2deg1.count()
# joinednode1points2deg1.take(5) 
# a,(b,c) means node a points to node b and c is 1/outdegree of node a

#swap first element and first element of second element which is a tuple
#a,(b,c) means node b points to node a and node a has degree c
    bpointsadega = joinednode1points2deg1.map(lambda x : (x[1][0], (x[0], x[1][1])))
# bpointsadega.take(5)
#doing groupByKey function
    rddgrouped = bpointsadega.groupByKey().mapValues(list)
# rddgrouped.count()
# rddgrouped.take(1) 
# [a,[(b,c),(d,e),..]] means b points to a and b has an 1/outdegree of c, d points to a and has e as 1/outdegree
    final = rddgrouped.sortByKey()
    num = final.count()
# print(num)
# final.take(2)
    qw = final.map(lambda x: (x[0],dict(x[1])))
# qw.take(1)
#m matrix
    m = qw.map(lambda x: np.array([x[1][i+1] if (i+1 in x[1].keys()) else 0 for i in range(num)]))
# m.take(3)
    size = m.count()
# type(size)
# print(size)
# r matrix
    r = np.full((size,), 1/size)
# r.shape
# r
    for i in range(iterations):
        prod = m.map(lambda x :np.dot(np.transpose(x), r))
    #     print(prod.take(1))
        prod1 = prod.map(lambda x: ((1-beta)/size) + (beta * x))
    #     print(prod1.take(1))
        r = np.array(prod1.collect())
    #     print(r)
        yu = np.argsort(r)
        a = yu[-5:]
        a.tolist()
        print('Iteration ',i+1,':')
        for j in range(4,-1,-1):
            print('\t',a[j] + 1, ': ', r[a[j]])
        print()
#         print("iter",i)
        
# Bottom five nodes
        if(i==39):
            print("Bottom five nodes for iteration 40")
            for i in range(5):
                print('\t',yu[i] + 1, ': ', r[yu[i]])
            print()
#Top 5 nodes
            print("Top five nodes for iteration 40")
            for i in range(4,-1,-1):
                print('\t',a[i] + 1, ': ', r[a[i]])

        

In [4]:
page_rank(dataset_path, iterations, beta)

Iteration  1 :
	 263 :  0.0018759018759018757
	 502 :  0.001866349206349206
	 126 :  0.001843125763125763
	 285 :  0.001819076479076479
	 146 :  0.0018094549894549896

Iteration  2 :
	 263 :  0.0020887152258490605
	 965 :  0.0019612553136778245
	 537 :  0.0018784860606765365
	 243 :  0.0018250404103778125
	 285 :  0.0018120241867479964

Iteration  3 :
	 263 :  0.0020238066893208664
	 537 :  0.0019302214828365047
	 965 :  0.0019265979884266352
	 243 :  0.0018556711996323288
	 285 :  0.001841159061219935

Iteration  4 :
	 263 :  0.0020124609096470636
	 537 :  0.0019423422859068046
	 965 :  0.0019272412154498733
	 243 :  0.001851426809420244
	 285 :  0.0018288144290373816

Iteration  5 :
	 263 :  0.002021301319496238
	 537 :  0.0019452806944060199
	 965 :  0.0019258364033716434
	 243 :  0.0018530462087332255
	 285 :  0.001827632665451319

Iteration  6 :
	 263 :  0.002020648241069512
	 537 :  0.001943229266241231
	 965 :  0.0019250388507970088
	 243 :  0.0018527946111016682
	 285 :  0.0018